In [1]:
#importing libraries and dependencies
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing data
dataset = load_dataset("imdb")

In [ ]:
#tokenizing data
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
# tokenized_dataset = dataset.map(tokenize_function, batched=True)
#save tokens
# from google.colab import drive
# drive.mount('/content/drive')
# tokenized_dataset.save_to_disk("/content/drive/MyDrive/tokenized_dataset")
# tokenized_dataset

In [3]:
#loading tokernizer
# from google.colab import drive
# drive.mount('/content/drive')
tokenized_dataset = load_from_disk("/content/drive/MyDrive/tokenized_dataset")

In [5]:
#splitting the data
train_valid = tokenized_dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_valid['train']
valid_dataset = train_valid['test']
test_dataset = tokenized_dataset['test']  # Use the official test set

#loading the model
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##Full Fine-tuning


In [7]:
#training
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=250,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    save_total_limit=1,
    seed=42,
    fp16=True,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

test_results = trainer.evaluate(test_dataset)
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")
#save model
save_path = "/content/drive/MyDrive/roberts-base_imdb_full-finetuned"

# Save model and tokenizer
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

/tmp/ipython-input-7-3073309799.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.182300,0.275338,0.932200
2,0.137900,0.210969,0.950200
3,0.091500,0.248517,0.951800
4,0.063900,0.251584,0.953400


Test Accuracy: 0.9554


('/content/drive/MyDrive/roberts-base_imdb_full-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/roberts-base_imdb_full-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/roberts-base_imdb_full-finetuned/vocab.json',
 '/content/drive/MyDrive/roberts-base_imdb_full-finetuned/merges.txt',
 '/content/drive/MyDrive/roberts-base_imdb_full-finetuned/added_tokens.json',
 '/content/drive/MyDrive/roberts-base_imdb_full-finetuned/tokenizer.json')

In [4]:
#load model
load_path = "/content/drive/MyDrive/roberts-base_imdb_full-finetuned"

# Load model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(load_path)

You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/roberts-base_imdb_full-finetuned and are newly initialized: ['classifier.bias', 'classifier.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attenti